In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# 1. Load the dataset
df = pd.read_csv("simulated_data/TAS_Airport_Data_2024_final.csv")



# 2. Handle Missing Values

# a) Impute numerical features with the mean
numerical_features = ["FlightDuration", "DepartureDelay", "ArrivalDelay", "PreviousFlightDelay", 
                      "AircraftTurnaroundTime", "CheckInQueueTime", "SecurityWaitTime",
                      "BaggageClaimBelts", "CustomsOfficers", "ImmigrationOfficers"]

numerical_imputer = SimpleImputer(strategy="mean")
df[numerical_features] = numerical_imputer.fit_transform(df[numerical_features])

# b) Impute categorical features with the most frequent value
categorical_features = ["ReasonForDelay", "GateAvailability", "GateType", "LoungeUsage"]

categorical_imputer = SimpleImputer(strategy="most_frequent")
df[categorical_features] = categorical_imputer.fit_transform(df[categorical_features])

# Justification: Handling missing values is crucial to avoid errors in subsequent steps. 
# We use imputation to fill in missing values with reasonable estimates.

# 3. Feature Engineering

# a) Create a new feature: TotalDelay
df["TotalDelay"] = df["DepartureDelay"] + df["ArrivalDelay"]

# b) Convert ScheduledTime and ScheduledArrivalTime to datetime objects
df["ScheduledTime"] = pd.to_datetime(df["ScheduledTime"])
df["ScheduledArrivalTime"] = pd.to_datetime(df["ScheduledArrivalTime"])

# c) Extract hour of the day from ScheduledTime and ScheduledArrivalTime
df["ScheduledHour"] = df["ScheduledTime"].dt.hour
df["ScheduledArrivalHour"] = df["ScheduledArrivalTime"].dt.hour

# Justification: Feature engineering creates new features or transforms existing ones 
# to improve model performance. We create a `TotalDelay` feature and extract 
# meaningful information from the datetime columns.

# 4. Feature Scaling and Encoding

# a) Define features for scaling and encoding
numerical_features_to_scale = ["FlightDuration", "TotalDelay", "PreviousFlightDelay", 
                               "AircraftTurnaroundTime", "CheckInQueueTime", "SecurityWaitTime",
                               "BaggageClaimBelts", "CustomsOfficers", "ImmigrationOfficers",
                               "ScheduledHour", "ScheduledArrivalHour"] 

categorical_features_to_encode = ["Airline", "WeatherCondition", "ReasonForDelay", 
                                   "GateAvailability", "GateType", "LoungeUsage"]

# b) Create transformers
numerical_transformer = Pipeline(steps=[("scaler", StandardScaler())])
categorical_transformer = Pipeline(steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))])

# c) Create a ColumnTransformer to apply transformations
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_features_to_scale),
        ("cat", categorical_transformer, categorical_features_to_encode),
    ]
)

# d) Fit and transform the data
df

FlightID Airline DepartureAirport ArrivalAirport OriginAirport  \
0      HY20240101_001D      HY              TAS            FRA           NaN   
1      KE20240101_002D      KE              TAS            KUL           NaN   
2      KE20240101_003D      KE              TAS            IST           NaN   
3      HY20240101_004D      HY              TAS            SIN           NaN   
4      HY20240101_005A      HY              NaN            NaN           FRA   
...                ...     ...              ...            ...           ...   
32935  TK20241231_086A      TK              NaN            NaN           DXB   
32936  HY20241231_087A      HY              NaN            NaN           DXB   
32937  HY20241231_088A      HY              NaN            NaN           LHR   
32938  TK20241231_089A      TK              NaN            NaN           DEL   
32939  SU20241231_090A      SU              NaN            NaN           ICN   

      Gate       ScheduledTime ScheduledArrivalTime  FlightDuration  \
0      C18 2024-01-01 22:11:00                  NaT           135.0   
1       C3 2024-01-01 08:02:00                  NaT           521.0   
2       C3 2024-01-01 20:33:00                  NaT           291.0   
3       C3 2024-01-01 20:03:00                  NaT           462.0   
4       C8                 NaT  2024-01-01 11:50:00           192.0   
...    ...                 ...                  ...             ...   
32935   B1                 NaT  2024-12-31 16:50:00           119.0   
32936  B16                 NaT  2024-12-31 22:10:00           591.0   
32937   C8                 NaT  2024-12-31 19:04:00           103.0   
32938  B20                 NaT  2025-01-01 06:42:00           596.0   
32939   C1                 NaT  2024-12-31 17:18:00           294.0   

       BaggageHandlers  ...  CheckInQueueTime  SecurityWaitTime  LoungeUsage  \
0                    4  ...               7.0              27.0           No   
1                    4  ...               7.0              27.0           No   
2                    4  ...               7.0              27.0           No   
3                    2  ...               7.0              27.0           No   
4                    4  ...               7.0              27.0           No   
...                ...  ...               ...               ...          ...   
32935                6  ...               7.0              27.0           No   
32936                2  ...               7.0              27.0           No   
32937                4  ...               7.0              27.0           No   
32938                4  ...               7.0              27.0           No   
32939                4  ...               7.0              27.0           No   

       OnTimePerformance BaggageClaimBelts  CustomsOfficers  \
0                Delayed               2.0              9.0   
1                Delayed               2.0              9.0   
2                Delayed               2.0              9.0   
3                Delayed               2.0              9.0   
4                Delayed               2.0              9.0   
...                  ...               ...              ...   
32935            Delayed               2.0              9.0   
32936            Delayed               2.0              9.0   
32937            Delayed               2.0              9.0   
32938            Delayed               2.0              9.0   
32939            Delayed               2.0              9.0   

      ImmigrationOfficers  TotalDelay  ScheduledHour  ScheduledArrivalHour  
0                     5.0         4.0           22.0                   NaN  
1                     5.0       123.0            8.0                   NaN  
2                     5.0         5.0           20.0                   NaN  
3                     5.0        48.0           20.0                   NaN  
4                     5.0         0.0            NaN                  11.0  
...                 